# Extracting and Loading the CSV

In [1]:
# importing libraries

import os
import sys
import pandas as pd

In [2]:
# loading scripts

sys.path.append(os.path.abspath("../scripts/python/"))
from loader import Loader
from extractor import Extractor

loader = Loader()
extractor = Extractor()

## Extract Data

In [3]:
# load csv into a dataframe
data = extractor.load_csv("../data/sample.csv")
data.head()

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Car; 48.85; 9.770344; 37.977391; 23.737688;...
1,2; Motorcycle; 98.09; 19.839417; 37.977642; 23...
2,3; Motorcycle; 63.80; 18.228752; 37.977997; 23...
3,4; Motorcycle; 145.72; 26.229014; 37.978135; 2...
4,5; Motorcycle; 138.01; 24.841425; 37.978134; 2...


In [4]:
# restructure data frame.
new_df = extractor.restructure(data)
new_df.head()

dataframe successfully created


,track_id,types,traveled_d,avg_speed,trajectory
0,1,Car,48.85,9.770344,"37.977391, 23.737688, 4.9178, 0.0518, -0.0299,..."
1,2,Motorcycle,98.09,19.839417,"37.977642, 23.737400, 16.9759, -0.0361, -0.022..."
2,3,Motorcycle,63.80,18.228752,"37.977997, 23.737264, 20.1906, -0.0795, -0.339..."
3,4,Motorcycle,145.72,26.229014,"37.978135, 23.737072, 2.7555, -0.0302, 0.0948,..."
4,5,Motorcycle,138.01,24.841425,"37.978134, 23.737103, 0.0000, 0.0000, 0.0000, ..."


## Load Into Warehouse

In [5]:
# create connection with wareouse
connection, cursor = loader.connect_to_server("localhost", "root", "localhost")

successfully connected


In [6]:
loader.create_db(cursor, "warehouse")

database successfully created


In [7]:
conn, cur = loader.connect_to_db("localhost", "root", "localhost", "warehouse")

successfully connected


In [8]:
loader.create_table(cur, "../scripts/sql/source_table.sql", "warehouse")

In [10]:
loader.insert_into_table(cur, conn, "warehouse", new_df, "source")

Data inserted successfully


In [11]:
loader.close_connection(cursor=cursor, connection=connection)

connection closed and transaction committed
